In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
import matplotlib.pylab as plt
# https://docs.scipy.org/doc/numpy/reference/routines.math.html
import numpy as np

In [4]:
from datetime import tzinfo, timedelta, datetime

In [5]:
from distutils.version import StrictVersion

In [6]:
import sklearn

assert StrictVersion(sklearn.__version__ ) >= StrictVersion('0.18.1')

sklearn.__version__

'0.18.1'

In [7]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

assert StrictVersion(tf.__version__) >= StrictVersion('1.1.0')

tf.__version__

'1.2.1'

In [8]:
import keras

assert StrictVersion(keras.__version__) >= StrictVersion('2.0.0')

keras.__version__

Using TensorFlow backend.


'2.0.6'

In [9]:
# !curl -O https://raw.githubusercontent.com/DJCordhose/speed-limit-signs/master/data/speed-limit-signs.zip
# !curl -O https://raw.githubusercontent.com/DJCordhose/speed-limit-signs/master/data/augmented-signs.zip    

In [10]:
# https://docs.python.org/3/library/zipfile.html
# from zipfile import ZipFile
# zip = ZipFile(r'speed-limit-signs.zip')
# zip.extractall('.')
# zip = ZipFile(r'augmented-signs.zip')
# zip.extractall('.')

In [11]:
# !ls -l speed-limit-signs

In [12]:
# !ls -l augmented-signs

In [13]:
import os
import skimage.data
import skimage.transform
from keras.utils.np_utils import to_categorical
import numpy as np

def load_data(data_dir, type=".ppm"):
    num_categories = 6

    # Get all subdirectories of data_dir. Each represents a label.
    directories = [d for d in os.listdir(data_dir) 
                   if os.path.isdir(os.path.join(data_dir, d))]
    # Loop through the label directories and collect the data in
    # two lists, labels and images.
    labels = []
    images = []
    for d in directories:
        label_dir = os.path.join(data_dir, d)
        file_names = [os.path.join(label_dir, f) for f in os.listdir(label_dir) if f.endswith(type)]
        # For each label, load it's images and add them to the images list.
        # And add the label number (i.e. directory name) to the labels list.
        for f in file_names:
            images.append(skimage.data.imread(f))
            labels.append(int(d))
    images64 = [skimage.transform.resize(image, (64, 64)) for image in images]
    y = np.array(labels)
    y = to_categorical(y, num_categories)
    X = np.array(images64)
    return X, y

In [14]:
# Depends on harware GPU architecture, set as high as possible (this works well on K80)
BATCH_SIZE = 500

In [15]:
# Load datasets.
ROOT_PATH = "./"

In [16]:
original_dir = os.path.join(ROOT_PATH, "speed-limit-signs")
original_images, original_labels = load_data(original_dir, type=".ppm")

In [17]:
data_dir = os.path.join(ROOT_PATH, "augmented-signs")
augmented_images, augmented_labels = load_data(data_dir, type=".png")

In [18]:
all_images = np.vstack((original_images, augmented_images))

In [19]:
all_labels = np.vstack((original_labels, augmented_labels))

In [20]:
# https://stackoverflow.com/a/4602224
p = numpy.random.permutation(len(all_labels))

In [21]:
p = numpy.random.permutation(len(all_labels))

In [22]:
shuffled_images = all_images[p]

In [23]:
shuffled_labels = all_labels[p]

In [24]:
# Turn this around if you want the large training set using augmented data or the original one

# X, y = original_images, original_labels
# X, y = augmented_images, augmented_labels
X, y = shuffled_images, shuffled_labels

In [25]:
# Same as above

# evaluation_X, evaluation_y = augmented_images, augmented_labels
evaluation_X, evaluation_y = original_images, original_labels

In [26]:
from sklearn.model_selection import train_test_split

In [27]:
checkpoint_callback = keras.callbacks.ModelCheckpoint('../tmp/model-checkpoints/weights.epoch-{epoch:02d}-val_loss-{val_loss:.2f}.hdf5');

In [28]:
early_stopping_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=30, verbose=1)

In [29]:
# https://github.com/tensorflow/tensorflow/blob/master/tensorflow/tensorboard/README.md
# https://keras.io/callbacks/#tensorboard
# http://stackoverflow.com/questions/42112260/how-do-i-use-the-tensorboard-callback-of-keras
tb_callback = keras.callbacks.TensorBoard(log_dir='../tmp/tf_log')
#                                          histogram_freq=1, write_graph=True, write_images=True)
#                                          histogram_freq=1, write_graph=True, write_images=True)
# tbCallBack = keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)
# To start tensorboard
# tensorboard --logdir=/mnt/c/Users/olive/Development/ml/tf_log
# open http://localhost:6006

In [30]:
# we want to distribute our different classes equally over test and train, this works using stratify
# https://github.com/amueller/scipy-2017-sklearn/blob/master/notebooks/04.Training_and_Testing_Data.ipynb
# http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [31]:
X_train.shape, y_train.shape

((3335, 64, 64, 3), (3335, 6))

In [32]:
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input
from keras.layers import Convolution2D, MaxPooling2D

# drop_out = 0.9
# drop_out = 0.75
drop_out = 0.5
# drop_out = 0.25
# drop_out = 0.0

# input tensor for a 3-channel 64x64 image
inputs = Input(shape=(64, 64, 3))

# one block of convolutional layers
x = Convolution2D(64, 3, 3, activation='relu')(inputs)
# x = Dropout(drop_out)(x)
x = Convolution2D(64, 3, 3, activation='relu')(x)
# x = Dropout(drop_out)(x)
x = Convolution2D(64, 3, 3, activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(drop_out)(x)

# one more block
x = Convolution2D(128, 3, 3, activation='relu')(x)
# x = Dropout(drop_out)(x)
x = Convolution2D(128, 3, 3, activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(drop_out)(x)

# one more block
x = Convolution2D(256, 3, 3, activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(drop_out)(x)

x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(drop_out)(x)

# softmax activation, 6 categories
predictions = Dense(6, activation='softmax')(x)
model = Model(input=inputs, output=predictions)
model.summary()
# model.compile(optimizer='rmsprop',
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 62, 62, 64)        1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 60, 64)        36928     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 58, 58, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 29, 29, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 29, 29, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 27, 27, 128)       73856     
__________

In [ ]:
!rm -rf ../tmp/tf_log
!rm -rf ../tmp/model-checkpoints

!mkdir ../tmp/model-checkpoints
!mkdir ../tmp/tf_log

In [ ]:
# Running on a GPU bach size might be critical depdendng on the GPU memory available
# more is desirable, but we might end up using 50 only 
print(datetime.utcnow().isoformat())
# BE CAREFUL, validation data is always the last data sets and not shuffled
# https://keras.io/getting-started/faq/#how-is-the-validation-split-computed
model.fit(X_train, y_train, epochs=1000, batch_size=BATCH_SIZE, validation_split=0.3, 
#           callbacks=[tb_callback, early_stopping_callback])
          callbacks=[tb_callback])
# model.fit(X_train, y_train, epochs=50, batch_size=200, validation_split=0.3)
print(datetime.utcnow().isoformat())

2017-07-31T13:48:46.738969
Train on 2334 samples, validate on 1001 samples
Epoch 1/1000
2334/2334 [==============================] - 15s - loss: 1.8009 - acc: 0.2035 - val_loss: 1.7851 - val_acc: 0.1968
Epoch 2/1000
2334/2334 [==============================] - 5s - loss: 1.7385 - acc: 0.2134 - val_loss: 1.7857 - val_acc: 0.1968
Epoch 3/1000
2334/2334 [==============================] - 5s - loss: 1.7185 - acc: 0.2219 - val_loss: 1.7880 - val_acc: 0.1978
Epoch 4/1000
2334/2334 [==============================] - 5s - loss: 1.7196 - acc: 0.2498 - val_loss: 1.7876 - val_acc: 0.1878
Epoch 5/1000
2334/2334 [==============================] - 5s - loss: 1.7091 - acc: 0.2678 - val_loss: 1.7848 - val_acc: 0.1978
Epoch 6/1000
2334/2334 [==============================] - 5s - loss: 1.7049 - acc: 0.2768 - val_loss: 1.7782 - val_acc: 0.2288
Epoch 7/1000
2334/2334 [==============================] - 5s - loss: 1.7097 - acc: 0.2669 - val_loss: 1.7772 - val_acc: 0.2078
Epoch 8/1000
2334/2334 [===========

2334/2334 [==============================] - 5s - loss: 0.8639 - acc: 0.6555 - val_loss: 2.3048 - val_acc: 0.3317
Epoch 128/1000
2334/2334 [==============================] - 5s - loss: 0.8530 - acc: 0.6530 - val_loss: 2.5344 - val_acc: 0.2987
Epoch 129/1000
2334/2334 [==============================] - 5s - loss: 0.8225 - acc: 0.6859 - val_loss: 2.6624 - val_acc: 0.2947
Epoch 130/1000
2334/2334 [==============================] - 5s - loss: 0.8324 - acc: 0.6718 - val_loss: 2.6191 - val_acc: 0.2897
Epoch 131/1000
2334/2334 [==============================] - 5s - loss: 0.8049 - acc: 0.6881 - val_loss: 2.1588 - val_acc: 0.3397
Epoch 132/1000
2334/2334 [==============================] - 5s - loss: 0.7697 - acc: 0.7014 - val_loss: 2.6757 - val_acc: 0.3037
Epoch 133/1000
2334/2334 [==============================] - 5s - loss: 0.7834 - acc: 0.6967 - val_loss: 3.2066 - val_acc: 0.2607
Epoch 134/1000
2334/2334 [==============================] - 5s - loss: 0.7940 - acc: 0.6962 - val_loss: 3.0269 -

2334/2334 [==============================] - 5s - loss: 0.3522 - acc: 0.8766 - val_loss: 3.3972 - val_acc: 0.3377
Epoch 254/1000
2334/2334 [==============================] - 5s - loss: 0.3753 - acc: 0.8650 - val_loss: 2.7588 - val_acc: 0.4216
Epoch 255/1000
2334/2334 [==============================] - 5s - loss: 0.3520 - acc: 0.8689 - val_loss: 2.7851 - val_acc: 0.3776
Epoch 256/1000
2334/2334 [==============================] - 5s - loss: 0.3428 - acc: 0.8753 - val_loss: 2.7765 - val_acc: 0.3806
Epoch 257/1000
2334/2334 [==============================] - 5s - loss: 0.3269 - acc: 0.8766 - val_loss: 2.6559 - val_acc: 0.4266
Epoch 258/1000
2334/2334 [==============================] - 5s - loss: 0.3168 - acc: 0.8775 - val_loss: 3.0905 - val_acc: 0.3596
Epoch 259/1000
2334/2334 [==============================] - 5s - loss: 0.3134 - acc: 0.8890 - val_loss: 2.6439 - val_acc: 0.4116
Epoch 260/1000
2334/2334 [==============================] - 5s - loss: 0.3194 - acc: 0.8817 - val_loss: 2.8608 -

2334/2334 [==============================] - 5s - loss: 0.1565 - acc: 0.9400 - val_loss: 3.2314 - val_acc: 0.4136
Epoch 380/1000
2334/2334 [==============================] - 5s - loss: 0.1504 - acc: 0.9473 - val_loss: 3.8831 - val_acc: 0.3876
Epoch 381/1000
2334/2334 [==============================] - 5s - loss: 0.1522 - acc: 0.9443 - val_loss: 3.5233 - val_acc: 0.3816
Epoch 382/1000
2334/2334 [==============================] - 5s - loss: 0.1683 - acc: 0.9460 - val_loss: 3.2028 - val_acc: 0.4286
Epoch 383/1000
2334/2334 [==============================] - 5s - loss: 0.1629 - acc: 0.9477 - val_loss: 3.0448 - val_acc: 0.4436
Epoch 384/1000
2334/2334 [==============================] - 5s - loss: 0.1741 - acc: 0.9387 - val_loss: 4.0782 - val_acc: 0.3546
Epoch 385/1000
2334/2334 [==============================] - 5s - loss: 0.2196 - acc: 0.9216 - val_loss: 3.4839 - val_acc: 0.3866
Epoch 386/1000
2334/2334 [==============================] - 5s - loss: 0.2224 - acc: 0.9177 - val_loss: 4.2527 -

2334/2334 [==============================] - 5s - loss: 0.1098 - acc: 0.9627 - val_loss: 4.4463 - val_acc: 0.3676
Epoch 506/1000
2334/2334 [==============================] - 5s - loss: 0.1028 - acc: 0.9653 - val_loss: 2.9582 - val_acc: 0.4705
Epoch 507/1000
2334/2334 [==============================] - 5s - loss: 0.1163 - acc: 0.9597 - val_loss: 4.1203 - val_acc: 0.3936
Epoch 508/1000
2334/2334 [==============================] - 5s - loss: 0.1203 - acc: 0.9623 - val_loss: 3.3668 - val_acc: 0.4216
Epoch 509/1000
2334/2334 [==============================] - 5s - loss: 0.1022 - acc: 0.9683 - val_loss: 4.1760 - val_acc: 0.3816
Epoch 510/1000
2334/2334 [==============================] - 5s - loss: 0.0979 - acc: 0.9627 - val_loss: 3.1496 - val_acc: 0.4605
Epoch 511/1000
2334/2334 [==============================] - 5s - loss: 0.0998 - acc: 0.9632 - val_loss: 5.0067 - val_acc: 0.3407
Epoch 512/1000
2334/2334 [==============================] - 5s - loss: 0.1290 - acc: 0.9554 - val_loss: 3.9340 -

In [ ]:
train_loss, train_accuracy = model.evaluate(X_train, y_train, batch_size=500)
train_loss, train_accuracy

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE)
test_loss, test_accuracy

In [ ]:
evaluation_loss, evaluation_accuracy = model.evaluate(evaluation_X, evaluation_y, batch_size=BATCH_SIZE)
evaluation_loss, evaluation_accuracy

In [ ]:
!mkdir models

In [ ]:
# model.save('models/conv-vgg-augmented.hdf5')
# model.save('models/conv-vgg-original.hdf5')
model.save('models/conv-vgg-mixed.hdf5')

In [ ]:
!ls -lh models

In [ ]:
# !curl --upload-file ./models/conv-vgg-augmented.hdf5 https://transfer.sh/conv-vgg-augmented.hdf5
!curl --upload-file ./models/conv-vgg-original.hdf5 https://transfer.sh/conv-vgg-original.hdf5